# Modelos
Estos son los modelos que se encargarán de procesar nuestro texto. En el caso de usar OpenAI, se tiene que crear una variable de ambiente llamada "OPENAI_API_KEY" con el API KEY de OpenAI.

Para hacer esto en linux se tiene que correr desde la terminal
```bash
export OPENAI_API_KEY="insertar_api_key_aqui"
```

O tambien se puede agregar a un archivo config.py que unicamente contiene la variable del API Key

```python
OPENAI_API_KEY="insertar_api_key_aqui"
```


In [ ]:
from langchain.llms import LlamaCpp, OpenAI 
import config
api = config.OPENAI_API_KEY 

In [ ]:
llm_openai = OpenAI(model_name = "text-davinci-003", openai_api_key=api)
#llm_llama = LlamaCpp(model_path="./llamacpp/models/7B/ggml-model-q4_0.bin")

In [ ]:
respuesta_openai = llm_openai("Hola, como estas?")
#respuesta_llama = llm_llama("Hola, como estas?")

In [ ]:
print(respuesta_openai)
#print(respuesta_llama)

## Modelos Chat
Los modelos que utilizamos eran para comletar texto, sin embargo podemos tambien usar modelos especificos para chat como ChatGPT

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [ ]:
chatgpt = ChatOpenAI(openai_api_key=api)

In [ ]:
respuesta = chatgpt([HumanMessage(content="Hola, como estas?")])
print(respuesta)

In [ ]:
respuesta

# Prompts
El prompt es el texto que se le envía al modelo similar a lo visto en la sección pasada. El éxito de los prompts está en que cuenten con una buena estructura y que den suficiente contexto el modelo para que entienda que esperamos de respuesta.
Cada prompt que le enviemos al modelo debe de tener una estructura similar, Langchain tiene un par de herramientas que nos facilitas esto.

In [ ]:
# En este template solo tenemos que poner el promt y entre llaves {} pones las variables
# que quieres que se reemplacen
from langchain import PromptTemplate

template_basico = """Eres un asistente virtual culinario que responde a preguntas
de manera muy breve.
Pregunta: Cuales son los ingredientes para preparar {platillo}
Respuesta:"""

In [ ]:
#Constuimos el template, especificandole cuales son las variables de entrada y cual es el texto que tiene que usar
prompt_temp = PromptTemplate(input_variables=["platillo"], template = template_basico)

#Aqui podemos ver como se reemplaza la variable platillo por tacos al pastor
promt_value = prompt_temp.format(platillo="tacos al pastor")
print(promt_value)

In [ ]:
respuesta_openai = llm_openai(promt_value)
print(respuesta_openai)

In [ ]:
# Se puede revisar el numero de tokens de un prompt en especifico
# Solo requieres instalar tiktoken con pip install tiktoken
llm_openai.get_num_tokens(promt_value)

# ChatPromptTemplates
AL igual que tenemos templates para modelos abiertos, LangChain tambien nos brinda templates para modelos de chat. Estos templates nos ayudan a darle la informacion a los modelos de chat en la manera en la que lo necesitan.

Los elementos de estos templates son:
* **Human**: El texto que escribimos nosotros
* **AI**: El texto que responde el modelo
* **System**: El texto que se le envía al modelo para darle contexto de su funcionamiento

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate

In [ ]:
# Armemos el template para el sistema
prompt_temp_sistema = PromptTemplate(
    template="Eres un asistente virtual que me recomienda una alternativa {adjetivo} a un producto",
    input_variables=["adjetivo"],
)

template_sistema = SystemMessagePromptTemplate(prompt=prompt_temp_sistema)


#Ahora para el humano
prompt_temp_humano = PromptTemplate(template="{texto}", input_variables=["texto"])

template_humano = HumanMessagePromptTemplate(prompt=prompt_temp_humano)


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([template_sistema, template_humano])

# Este es el formato del prompt que acabamos de armar
chat_promt_value = chat_prompt.format_prompt(adjetivo="economica", texto="ipad").to_messages()
print(chat_promt_value)

In [ ]:
chat_resp = chatgpt(chat_promt_value)
print(chat_resp)


# Example selector
Cuando estamos usando un modelo probablemente queremos darle un par de ejemplos para influir en el tipo de respuesta que nos brinda, con example selector podemos hacer esto de manera sencilla.

In [ ]:
from langchain import FewShotPromptTemplate

# Primero hacemos una lista de  ejemplos de los que queremos que el modelo aprenda
ejemplos = [
    {"pregunta": "¿Cuál es el ingrediente principal de la pizza?", "respuesta": "La masa y salsa de tomate"},
    {"pregunta": "¿Cuál es el ingrediente principal de la hamburguesa?", "respuesta": "La carne y el pan"},
    {"pregunta": "¿Cuál es el ingrediente principal del burrito?", "respuesta": "La tortilla y la carne"}
]

# Ahora armamos un template para el modelo, como en los ejemplos incluimos respuesta en el template debe de ir un espacio para esta variable.
# En template le decimos como queremos aque formatee el promt y de donde obtener las variables dentro de la variable ejemplos
promt_temp_ejemplos = PromptTemplate(input_variables=["pregunta", "respuesta"], 
                                     template = "Pregunta: {pregunta}\nRespuesta: {respuesta}")

In [ ]:
promt_ejemplos = FewShotPromptTemplate(example_prompt=promt_temp_ejemplos, 
                                       examples=ejemplos, 
                                       prefix = "Eres un asistenet virtual culinario que responde preguntas de manera muy breve",
                                       suffix = "Pregunta: {pregunta}\nRespuesta:", 
                                        input_variables=["pregunta"]) 

In [ ]:
prompt_value = promt_ejemplos.format(pregunta="¿Cuál es el ingrediente principal del coctel de camaron?")
print(prompt_value)


In [ ]:
respuesta_ingredientes = llm_openai(prompt_value)
print(respuesta_ingredientes)

In [ ]:
llm_openai("¿Cuál es el ingrediente principal del coctel de camaron?")

In [ ]:
llm_openai.get_num_tokens(prompt_value)

# Output parser
LangChain tambien nos da la oportunidad de parsear o formatear las respuestas que nos da el modelo de alguna manera que nos sea mas útil.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()


In [ ]:
format_instructions

In [ ]:
template_basico_parser = """Cuales son los ingredientes para preparar {platillo}\n{como_parsear}"""

In [ ]:
prompt_temp_parser = PromptTemplate(input_variables=["platillo"], 
                                    template = template_basico_parser, 
                                    partial_variables={"como_parsear": format_instructions})




In [ ]:
promt_value_parser = prompt_temp_parser.format(platillo="tacos al pastor")

In [ ]:
print(promt_value_parser)

In [ ]:
respuesta_parser = llm_openai(promt_value_parser)

In [ ]:
respuesta_parser

In [ ]:
output_parser.parse(respuesta_parser)